# Leveraging Pre-trained Language Models for Improved Low Resource Machine Translation

# Presenter: Professor Vukosi Marivate

# Introduction:

You are welcome to the ABSA Masterclass: News Classification Hands-on Workshop!

In this hands-on tutorial, we will be learning/practicing the use of pre-trained language models (PLMs) to train an XLMR-base model for news classification.

This notebook was prepared using the Masakhane-News code:  https://github.com/masakhane-io/masakhane-news.git

# Topics:

Content: Low Resource News Classification/ Categorization

Level: Beginner to Intermediate

# Aims/Learning Objectives:

    Setting up the training pipeline for fine-tuning a XLMR base for News Classification

# Prerequisites:

     Jupyter Notebook 

# Outline:

    Installation and importing of libraries
    Setting up the necessary training classes and modules
    Setting up training paths and hyper-parameters
    Running training and evaluation
    Trying single examples
    Testing with zero-shot 

# Before you start:

It is strongly advised that you use a GPU to speed up training.


In [62]:
import argparse
import glob
import logging
import os
import random

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer, AutoModel
from tqdm import tqdm, trange
from util_textclass import get_labels, read_instances_from_file, convert_instances_to_features_and_labels
import sklearn.metrics
import argparse
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    BertConfig,
    BertForSequenceClassification,
    BertTokenizer,
    DistilBertConfig,
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    RobertaConfig,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    XLMRobertaConfig,
    XLMRobertaForSequenceClassification,
    XLMRobertaTokenizer,
    get_linear_schedule_with_warmup,
)

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter


logger = logging.getLogger(__name__)

In [63]:
class Instance:

    def __init__(self, text, label):
        self.text = text
        self.label = label

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids, label):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.label = label

def read_instances_from_file(data_dir, mode, delimiter="\t"):
    file_path = os.path.join(data_dir, "{}.tsv".format(mode))
    instances = []

    df = pd.read_csv(file_path, sep='\t')
    N = df.shape[0]

    for i in range(N):
        instances.append(Instance(df['headline'].iloc[i] +' '+df['text'].iloc[i], df['category'].iloc[i]))
    '''
    with open(file_path, "r", encoding='utf-8') as input_file:
        line_data = input_file.read()

    line_data = line_data.splitlines()
    for l, line in enumerate(line_data):
        if l==0:
            continue
        else:
            text_vals = line.strip().split(delimiter)
            text, label = ' '.join(text_vals[:-1]), text_vals[-1]
            instances.append(Instance(text, label))
    '''

    return instances

def convert_instances_to_features_and_labels(instances, tokenizer, labels, max_seq_length):
    label_map = {label: i for i, label in enumerate(labels)}

    features = []
    for instance_idx, instance in enumerate(instances):
        tokenization_result = tokenizer.encode_plus(text=instance.text,
                                                    max_length=max_seq_length, pad_to_max_length=True,
                                                     truncation=True)
        token_ids = tokenization_result["input_ids"]
        try:
            token_type_ids = tokenization_result["token_type_ids"]
        except:
            token_type_ids = None
        attention_masks = tokenization_result["attention_mask"]

        if instance.label not in label_map:
            continue
        label = label_map[instance.label]

        if "num_truncated_tokens" in tokenization_result:
            logger.info(f"Removed {tokenization_result['num_truncated_tokens']} tokens from {instance.text} as they "
                         f"were longer than max_seq_length {max_seq_length}.")

        if instance_idx < 3:
            logger.info("Tokenization example")
            logger.info(f"  text: {instance.text}")
            logger.info(f"  tokens (by input): {tokenizer.tokenize(instance.text)}")
            logger.info(f"  token_ids: {tokenization_result['input_ids']}")
            #logger.info(f"  token_type_ids: {tokenization_result['token_type_ids']}")
            logger.info(f"  attention mask: {tokenization_result['attention_mask']}")

        features.append(
            InputFeatures(input_ids=token_ids, attention_mask=attention_masks, token_type_ids=token_type_ids, label=label)
        )

    return features

def get_labels(path):
    if path:
        with open(path, "r") as f:
            labels = f.read().splitlines()
        return labels
    else:
        return  ['sports', 'health', 'technology', 'business', 'politics', 'entertainment', 'religion', 'uncategorized']

In [64]:
MODEL_CLASSES = {
    "bert": (BertConfig, BertForSequenceClassification, BertTokenizer),
    "roberta": (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer),
    "distilbert": (DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer),
    "xlmroberta": (XLMRobertaConfig, XLMRobertaForSequenceClassification, XLMRobertaTokenizer),
}

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

In [72]:
def train(args, train_dataset, dev_dataset, labels, model, tokenizer):
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    '''
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer__.pt")) and os.path.isfile(
        os.path.join(args.model_name_or_path, "scheduler__.pt")
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))
    '''

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    '''
    if os.path.exists(args.output_dir): #model_name_or_path):
        # set global_step to gobal_step of last saved checkpoint from model path
        global_step = int(args.model_name_or_path.split("-")[-1].split("/")[0])
        epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
        steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

        logger.info("  Continuing training from checkpoint, will skip to saved global_step")
        logger.info("  Continuing training from epoch %d", epochs_trained)
        logger.info("  Continuing training from global step %d", global_step)
        logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
    '''
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproductibility

    eval_fones = []
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}
            if args.model_type not in ["distilbert", "xlmroberta"]:
                inputs["token_type_ids"] = (
                    batch[2] if args.model_type in ["bert"] else None
                )  # XLM and DistilBERT don't use segment_ids
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1


            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break

        # EVALUATE + EARLY STOPPING
        if global_step > 500 and args.n_gpu == 1: # and global_step % args.save_steps == 0:
            eval_results, _ = evaluate(args, model, tokenizer, labels, "dev", display_res=True)
            f1_step = round(eval_results["acc"], 5)
            eval_fones.append(f1_step)
            print("eval result: ", global_step, f1_step)

            #n = 1
            #while len(eval_fones) > 5 and n < 6 and eval_fones[-1] < eval_fones[-1 * (n + 1)]:
            #    print("no: ", n)
            #    n += 1

            #if n == 6:
            #    logger.info("EARLY STOPPING ..... ")
            #    break

            max_f1 = max(eval_fones)

            if f1_step == max_f1:
                output_dir = args.output_dir
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                model_to_save = (
                    model.module if hasattr(model, "module") else model
                )  # Take care of distributed/parallel training
                model_to_save.save_pretrained(output_dir)
                tokenizer.save_pretrained(output_dir)

                torch.save(args, os.path.join(output_dir, "training_args.bin"))
                logger.info("Saving model checkpoint to %s", output_dir)

                #torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                #torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                #logger.info("Saving optimizer and scheduler states to %s", output_dir)


        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break



    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step


def evaluate(args, model, tokenizer, labels, mode, prefix="", display_res=False):
    # load_and_cache_examples(args, tokenizer, labels, mode='train')
    eval_dataset = load_and_cache_examples(args, tokenizer, labels, mode)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # multi-gpu eval
    #if args.n_gpu > 1:
    #    model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}
            if args.model_type != "distilbert":
                inputs["token_type_ids"] = (
                    batch[2] if args.model_type in ["bert"] else None
                )  # XLM and DistilBERT don't use segment_ids
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds, axis=1)
    eval_report = sklearn.metrics.classification_report(out_label_ids, preds,
                                                        labels=range(len(labels)),
                                                        target_names=labels,
                                                        output_dict=True
                                                        )

    results = {
        "loss": eval_loss,
        "precision": eval_report["weighted avg"]["precision"],
        "recall": eval_report["weighted avg"]["recall"],
        "f1": eval_report["weighted avg"]["f1-score"],
        "acc": sklearn.metrics.accuracy_score(out_label_ids, preds),
    }

    if not display_res:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(results.keys()):
            logger.info("  %s = %s", key, str(results[key]))

    return results, preds

def inference_eval( headlines, text,text_class, args, model, tokenizer, labels, mode, prefix="", display_res=False):
        # load_and_cache_examples(args, tokenizer, labels, mode='train')
        eval_dataset   =  load_inference_examples(args, headlines, text, text_class,  tokenizer, labels, mode=mode)
        args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
        # Note that DistributedSampler samples randomly
        eval_sampler = SequentialSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)
    
        # multi-gpu eval
        #if args.n_gpu > 1:
        #    model = torch.nn.DataParallel(model)
    
        # Eval!
        logger.info("***** Running evaluation {} *****".format(prefix))
        logger.info("  Num examples = %d", len(eval_dataset))
        logger.info("  Batch size = %d", args.eval_batch_size)
        print("  Num examples = %d", len(eval_dataset))
        print("  Batch size = %d", args.eval_batch_size)
        
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            batch = tuple(t.to(args.device) for t in batch)
    
            with torch.no_grad():
                inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}
                if args.model_type != "distilbert":
                    inputs["token_type_ids"] = (
                        batch[2] if args.model_type in ["bert"] else None
                    )  # XLM and DistilBERT don't use segment_ids
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]
    
                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs["labels"].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)
    
        eval_loss = eval_loss / nb_eval_steps
        preds = np.argmax(preds, axis=1)
        eval_report = sklearn.metrics.classification_report(out_label_ids, preds,
                                                            labels=range(len(labels)),
                                                            target_names=labels,
                                                            output_dict=True
                                                            )
    
        results = {
            "loss": eval_loss,
            "precision": eval_report["weighted avg"]["precision"],
            "recall": eval_report["weighted avg"]["recall"],
            "f1": eval_report["weighted avg"]["f1-score"],
            "acc": sklearn.metrics.accuracy_score(out_label_ids, preds),
        }
    
        if not display_res:
            logger.info("***** Eval results {} *****".format(prefix))
            for key in sorted(results.keys()):
                logger.info("  %s = %s", key, str(results[key]))
    
        return results, preds

def load_and_cache_examples(args, tokenizer, labels, mode='train'):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Load data features from cache or dataset file
    cached_features_file = os.path.join(
        args.data_dir,
        "cached_{}_{}_{}".format(mode,
            list(filter(None, args.model_name_or_path.split("/"))).pop(),
            str(args.max_seq_length)
        ),
    )
    if os.path.exists(cached_features_file) and not args.overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        instances = read_instances_from_file(args.data_dir, mode)
        features = convert_instances_to_features_and_labels(instances, tokenizer, labels, args.max_seq_length)
        if args.local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)

    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    if args.model_type != 'xlmroberta':
        all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    else:
        all_token_type_ids = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
    return dataset

def load_inference_examples(args, headlines, text,text_class,  tokenizer, labels, mode='test'):
        if args.local_rank not in [-1, 0] and not evaluate:
            torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache
            # Load data features from cache or dataset file
        cached_features_file = os.path.join(
            args.data_dir,
            "cached_{}_{}_{}".format('infer',
                list(filter(None, args.model_name_or_path.split("/"))).pop(),
                str(args.max_seq_length)
            ),
        )        
        
        instances = []
        # instances.append(Instance(df['headline'].iloc[i] +' '+df['text'].iloc[i], df['category'].iloc[i]))
        instances.append(Instance(headline +' '+ text, text_class))
        # covert to features
        features = convert_instances_to_features_and_labels(instances, tokenizer, labels, args.max_seq_length)
        if args.local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)
            
        if args.local_rank == 0 and not evaluate:
                torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache
        
        # Convert to Tensors and build dataset
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
        if args.model_type != 'xlmroberta':
            all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
        else:
            all_token_type_ids = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
        all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
        
        dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
        return dataset
            
    

In [73]:
def get_args():
    """
    Get training arguments
    """
    parser = argparse.ArgumentParser()
    # Required parameters
    parser.add_argument(
        "--data_dir",
        default=None,
        type=str,
        help="The input data dir. Should contain the training files for the CoNLL-2003 NER task.",
    )
    parser.add_argument(
        "--model_type",
        default=None,
        type=str,
        #help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--model_name_or_path",
        default=None,
        type=str,
        help="Path to pre-trained model or shortcut name selected in the list: " + ", ",
    )
    parser.add_argument(
        "--input_dir",
        default=None,
        type=str,
        required=False,
        help="The input model directory.",
    )
    parser.add_argument(
        "--output_dir",
        default=None,
        type=str,
        help="The output directory where the model predictions and checkpoints will be written.",
    )

    # Other parameters
    parser.add_argument(
        "--labels",
        default="",
        type=str,
        help="Path to a file containing all labels. If not specified, CoNLL-2003 labels are used.",
    )
    parser.add_argument(
        "--config_name", default="", type=str, help="Pretrained config name or path if not the same as model_name"
    )
    parser.add_argument(
        "--tokenizer_name",
        default="",
        type=str,
        help="Pretrained tokenizer name or path if not the same as model_name",
    )
    parser.add_argument(
        "--cache_dir",
        default="",
        type=str,
        help="Where do you want to store the pre-trained models downloaded from s3",
    )
    parser.add_argument(
        "--max_seq_length",
        default=128,
        type=int,
        help="The maximum total input sequence length after tokenization. Sequences longer "
        "than this will be truncated, sequences shorter will be padded.",
    )
    parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_finetune", action="store_true", help="Whether to run training.")
    parser.add_argument("--do_eval", action="store_true", help="Whether to run eval on the dev set.")
    parser.add_argument("--do_predict", action="store_true", help="Whether to run predictions on the test set.")
    parser.add_argument(
        "--evaluate_during_training",
        action="store_true",
        help="Whether to run evaluation during training at each logging step.",
    )
    parser.add_argument(
        "--do_lower_case", action="store_true", help="Set this flag if you are using an uncased model."
    )

    parser.add_argument("--per_gpu_train_batch_size", default=8, type=int, help="Batch size per GPU/CPU for training.")
    parser.add_argument(
        "--per_gpu_eval_batch_size", default=8, type=int, help="Batch size per GPU/CPU for evaluation."
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument(
        "--num_train_epochs", default=3.0, type=float, help="Total number of training epochs to perform."
    )
    parser.add_argument(
        "--max_steps",
        default=-1,
        type=int,
        help="If > 0: set total number of training steps to perform. Override num_train_epochs.",
    )
    parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")

    parser.add_argument("--logging_steps", type=int, default=500, help="Log every X updates steps.")
    parser.add_argument("--save_steps", type=int, default=500, help="Save checkpoint every X updates steps.")
    parser.add_argument(
        "--eval_all_checkpoints",
        action="store_true",
        help="Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number",
    )
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument(
        "--overwrite_output_dir", action="store_true", help="Overwrite the content of the output directory"
    )
    parser.add_argument(
        "--overwrite_cache", action="store_true", help="Overwrite the cached training and evaluation sets"
    )
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")

    parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
    parser.add_argument("--server_ip", type=str, default="", help="For distant debugging.")
    parser.add_argument("--server_port", type=str, default="", help="For distant debugging.")

    return parser.parse_known_args()

In [74]:
data_path = "./data/xho"
args, _ = get_args()
args.data_dir = data_path  # to-change: supply data directory
args.output_dir = "../../../../../ext_data/neo/Masakha-news-models/" # to-change: supply output directory
args.model_type = "xlmroberta" #"bert"
args.model_name_or_path = "xlm-roberta-base" #"bert-base-multilingual-cased"
args.max_seq_length = 200
args.output_result = "test_result"
args.output_prediction_file = "test_prediction"
args.num_train_epochs = 2
args.per_gpu_train_batch_size = 2
args.save_steps = 10000
args.seed = 1
args.do_train = True
args.do_eval = True
args.do_predict = True

In [17]:
def main(args):
    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup distant debugging if needed
    if args.server_ip and args.server_port:
        # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd

        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()

    # Setup CUDA, GPU & distributed training
    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        args.n_gpu = torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        torch.distributed.init_process_group(backend="nccl")
        args.n_gpu = 1
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s,",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
    )

    # Set seed
    set_seed(args)

    labels = get_labels(args.labels)
    num_labels = len(labels)


    # Load pretrained model and tokenizer
    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    args.model_type = args.model_type.lower()
    config_class, model_class, tokenizer_class = AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
    config = config_class.from_pretrained(
        args.config_name if args.config_name else args.model_name_or_path,
        num_labels=num_labels,
        id2label={str(i): label for i, label in enumerate(labels)},
        label2id={label: i for i, label in enumerate(labels)},
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    tokenizer = tokenizer_class.from_pretrained(
        args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
        do_lower_case=args.do_lower_case,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    model = model_class.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )

    if args.local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, labels, mode="train")
        dev_dataset = load_and_cache_examples(args, tokenizer, labels, mode="dev")
        global_step, tr_loss = train(args, train_dataset, dev_dataset, labels, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)


    # Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
    if args.do_train and (args.local_rank == -1 or torch.distributed.get_rank() == 0):
        # Create output directory if needed
        output_dir = args.output_dir
        if not os.path.exists(output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(output_dir)

        logger.info("Saving model checkpoint to %s", output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        #model = model_class.from_pretrained(args.output_dir)
        #tokenizer = tokenizer_class.from_pretrained(args.output_dir)
        #model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = model_class.from_pretrained(checkpoint)
            model.to(args.device)
            result, _ = evaluate(args, model, tokenizer, labels, mode='dev', prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)


    if args.do_predict and args.local_rank in [-1, 0]:
        tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
        model = model_class.from_pretrained(args.output_dir)
        model.to(args.device)
        result, predictions = evaluate(args, model, tokenizer, labels, mode='test')
        predictions = list(predictions)
        id2label = {str(i): label for i, label in enumerate(labels)}

        # Save results
        output_test_results_file = os.path.join(args.output_dir, args.output_result+".txt")
        with open(output_test_results_file, "w") as writer:
            for key in sorted(result.keys()):
                writer.write("{} = {}\n".format(key, str(result[key])))

        output_test_predictions_file = os.path.join(args.output_dir, args.output_prediction_file+".txt")
        with open(output_test_predictions_file, "w", encoding='utf-8') as writer:
            df = pd.read_csv(os.path.join(args.data_dir, "test.tsv"), sep='\t')
            N = df.shape[0]

            texts = list(df['headline'].values)
            for i in range(N):
                output_line = texts[i] + "\t" + id2label[str(predictions[i])] + "\n"
                writer.write(output_line)
            '''
            with open(os.path.join(args.data_dir, "test.tsv"), "r", encoding='utf-8') as f:
                line_data = f.read()
            line_data =  line_data.splitlines()
            for l, line in enumerate(line_data):
                if l == 0:
                    continue
                else:
                    text_vals = line.strip().split("\t")
                    if len(text_vals) < 2: text_vals += [7]
                    text, label = text_vals
                    output_line = text + "\t" + id2label[str(predictions[l-1])] + "\n"
                    writer.write(output_line)
            '''

    return results


if __name__ == "__main__":
      main(args)

07/27/2024 09:07:52 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 2, distributed training: False,
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
07/27/2024 09:07:55 - INFO - __main__ -   Training/evaluation parameters Namespace(data_dir='./data/xho', model_type='xlmroberta', model_name_or_path='xlm-roberta-base', input_dir=None, output_dir='../../../../../ext_data/neo/Masakha-news-models/', labels='', config_name='', tokenizer_name='', cache_dir='', max_seq_length=200, do_train=True, do_finetune=False, do_eval=True, do_predict=True, evaluate_during_training=False, do_lower_case=False, per_gpu_train_batch_size=2, per_gpu_eval_batch_size=8, gradient_accum

# Example

In [75]:
# Text to test goes here
text_class = 'uncategorized'
headline = "Iinguqu kwiRadio 2000"
text = "Ukufakwa kwesicelo senkxaso-mali yonyaka ka-2023, yeNkqubo yeSizwe yoNcedo lwezeMali (i-NSFAS) sele kuvuliwe.UMphathiswa weMfundo ePhakamileyo."
print(text)

Ukufakwa kwesicelo senkxaso-mali yonyaka ka-2023, yeNkqubo yeSizwe yoNcedo lwezeMali (i-NSFAS) sele kuvuliwe.UMphathiswa weMfundo ePhakamileyo.


In [76]:
# Setup CUDA, GPU & distributed training
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend="nccl")
    args.n_gpu = 1
args.device = device
args.per_gpu_eval_batch_size  = 1
args.per_gpu_train_batch_size = 1
args.eval_batch_size = 1
args.model_type = args.model_type.lower()
labels  =  ['sports', 'health', 'technology', 'business', 'politics', 'entertainment', 'religion', 'uncategorized']
id2label = {str(i): label for i, label in enumerate(labels)}
config_class, model_class, tokenizer_class = AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
model = model_class.from_pretrained(args.output_dir)
model.to(args.device)
result, predictions = inference_eval(headline, text, text_class, args, model, tokenizer, labels, mode='test')
predictions = list(predictions)

# Save results
# output_test_results_file = os.path.join(args.output_dir, args.output_result+".txt")
# with open(output_test_results_file, "w") as writer:
#     for key in sorted(result.keys()):
#         writer.write("{} = {}\n".format(key, str(result[key])))
output_line = headline + text + "\t" + id2label[str(predictions[0])] + "\n"
print(output_line)

/home/neo/anaconda3/envs/google_env_nli/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
07/27/2024 10:30:26 - INFO - __main__ -   Tokenization example
07/27/2024 10:30:26 - INFO - __main__ -     text: Iinguqu kwiRadio 2000 Ukufakwa kwesicelo senkxaso-mali yonyaka ka-2023, yeNkqubo yeSizwe yoNcedo lwezeMali (i-NSFAS) sele kuvuliwe.UMphathiswa weMfundo ePhakamileyo.
07/27/2024 10:30:26 - INFO - __main__ -     tokens (by input): ['▁I', 'ingu', 'qu', '▁kwi', 'Radio', '▁2000', '▁Uku', 'fa', 'kwa', '▁kwe', 'sic', 'elo', '▁sen', 'k'

  Num examples = %d 1
  Batch size = %d 2


Evaluating: 100%|████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 181.21it/s]
/home/neo/anaconda3/envs/google_env_nli/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/neo/anaconda3/envs/google_env_nli/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/neo/anaconda3/envs/google_env_nli/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predic

Iinguqu kwiRadio 2000Ukufakwa kwesicelo senkxaso-mali yonyaka ka-2023, yeNkqubo yeSizwe yoNcedo lwezeMali (i-NSFAS) sele kuvuliwe.UMphathiswa weMfundo ePhakamileyo.	entertainment



# Zero shot

In [23]:
args, _ = get_args()
args.data_dir = "./data/sna"  # to-change: supply data directory for zero shot
args.output_dir = "../../../../../ext_data/neo/Masakha-news-models/" # to-change: supply output directory
args.model_type = "xlmroberta" #"bert"
args.model_name_or_path = "../../../../../ext_data/neo/Masakha-news-models/" #"bert-base-multilingual-cased"
args.max_seq_length = 200
args.num_train_epochs = 2
args.per_gpu_train_batch_size = 2
args.output_result = "_" + args.data_dir.split("/")[-1] + "_transfer_result"
args.output_prediction_file = "_" + args.data_dir.split("/")[-1] + "_transfer_prediction"
args.save_steps = 10000
args.seed = 1
args.do_train = False
args.do_eval = False
args.do_predict = True

In [24]:
# main with zero shot
main(args)

07/27/2024 09:19:45 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 2, distributed training: False,
07/27/2024 09:19:46 - INFO - __main__ -   Training/evaluation parameters Namespace(data_dir='./data/sna', model_type='xlmroberta', model_name_or_path='../../../../../ext_data/neo/Masakha-news-models/', input_dir=None, output_dir='../../../../../ext_data/neo/Masakha-news-models/', labels='', config_name='', tokenizer_name='', cache_dir='', max_seq_length=200, do_train=False, do_finetune=False, do_eval=False, do_predict=True, evaluate_during_training=False, do_lower_case=False, per_gpu_train_batch_size=2, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2, max_steps=-1, warmup_steps=0, logging_steps=500, save_steps=10000, eval_all_checkpoints=False, no_cuda=False, overwrite_output_dir=False, overwrite_cache=False, seed=1, local_rank=-1, server_ip='', server_port='', outpu

{}

# Zero shot one example

In [ ]:
# Text to test goes here
text_class = 'uncategorized'
headline = ""
text = ""
print(text)

In [ ]:
# Setup CUDA, GPU & distributed training
if args.local_rank == -1 or args.no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend="nccl")
    args.n_gpu = 1
args.device = device
args.per_gpu_eval_batch_size  = 1
args.per_gpu_train_batch_size = 1
args.eval_batch_size = 1
args.model_type = args.model_type.lower()
labels  =  ['sports', 'health', 'technology', 'business', 'politics', 'entertainment', 'religion', 'uncategorized']
id2label = {str(i): label for i, label in enumerate(labels)}
config_class, model_class, tokenizer_class = AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
model = model_class.from_pretrained(args.output_dir)
model.to(args.device)
result, predictions = inference_eval(headline, text, text_class, args, model, tokenizer, labels, mode='test')
predictions = list(predictions)

# Save results
# output_test_results_file = os.path.join(args.output_dir, args.output_result+".txt")
# with open(output_test_results_file, "w") as writer:
#     for key in sorted(result.keys()):
#         writer.write("{} = {}\n".format(key, str(result[key])))
output_line = headline + text + "\t" + id2label[str(predictions[0])] + "\n"
print(output_line)